# Lesson Notebook 10 - Embedding-Based Retrieval

In this notebook, we'll explore retrieving and ranking news headlines in response to a query. We'll use an encoder model similar to the Universal Sentence Encoder, to create vectors for each headline and then a vector for the query.  We'll use a library called [SentenceTransformers](https://www.sbert.net/) that has a large number of underlying model weight sets on Hugging Face.  Sentence Transformers are designed to take a sequence of words like a sentence as input and generates an representative vector, an embedding, as output. Note that Sentence Transformers are only available in PyTorch but that won't affect our use here thanks to the HuggingFace API.

First, we'll generate vectors for our headlines and hold those. Then we'll generate an embedding for our query and we'll just use Nearest Neighbors search on the full set of news headlines. Finally, we'll cluster the news headline embeddings first, and only apply Nearest Neighbors to the top k clusters whose centroids are most similar to the query embedding.

If we were trying to build a system that needed to scale,we would use something like the ScaNN library to hold our embeddings and perform our searches.

<a id = 'returnToTop'></a>

## Notebook Contents

  * 1. [Setup](#setup)
  * 2. [Data Preparation](#dataPrep)
  * 3. [Encode Embeddings](#encodeData)
  * 4. [Query and Retrieval](#queryRet)
  * 5. [Retrieval via Clusters](#clusterRet)
  * 6. [Answers](#answers)      









[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datasci-w266/2025-fall-main/blob/master/materials/lesson_notebooks/lesson_10_embedding_based_retrieval.ipynb)

[Return to Top](#returnToTop)  
<a id = 'setup'></a>

### 1. Setup

In [1]:
!pip install -q -U sentence-transformers
!pip install -q -U datasets==3.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.7 MB/s eta 0:00:00


In [2]:
import os
import time
import numpy as np
from datasets import load_dataset

from scipy.spatial.distance import cosine
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

[Return to Top](#returnToTop)  
<a id = 'dataPrep'></a>

### 2. Data Preparation

For our data we'll use the test portion of the XSum sumarization data set.  The goal of XSum is to generate a one line summary of the input article.  We'll grab the 'summary' field as this will be an excellent set of "sentences" for our retrieval experiment.  It takes about a minute to process the data and get us the test records.

In [3]:
dataset = load_dataset("EdinburghNLP/xsum", split="test")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

xsum.py: 0.00B [00:00, ?B/s]

default/train/0000.parquet:   0%|          | 0.00/304M [00:00<?, ?B/s]

default/validation/0000.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

default/test/0000.parquet:   0%|          | 0.00/17.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [4]:
len(dataset)

11334

In [5]:
dataset[0]

{'document': 'Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.\nWorkers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.\nThe Welsh Government said more people than ever were getting help to address housing problems.\nChanges to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.\nPrison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.\nHowever, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.\nAndrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the

[Return to Top](#returnToTop)  
<a id = 'encodeData'></a>

### 3. Encode embeddings

We'll load the sentence transformers with a smaller model so that it can run quickly in the live session.  You can experiment with others to see the tradeoff between size, processing time, and quality.  For example, you could load the sentence transformer with `'sentence-transformers/all-roberta-large-v1'` and leverage the improvements that come with using a large RoBERTa model.  You can see [a full listing of models](https://huggingface.co/models?library=sentence-transformers&sort=downloads) at HuggingFace.  We'll use the checkpoint based on [this paper](https://aclanthology.org/2022.acl-long.62.pdf).

In [6]:
encoder_model = SentenceTransformer('sentence-transformers/LaBSE')

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [7]:
# Just encoding a subset so it doesn't take too long during the live session (24 seconds)

news_headlines = [x['summary'] for x in dataset]
news_embeddings = encoder_model.encode(news_headlines[:7500])

[Return to Top](#returnToTop)  
<a id = 'queryRet'></a>

### 4. Query and Retrieval

First, we'll need to create a query and generate an embedding to represent it. Then we'll use that query embedding to walk through *all* of the headline embeddings and find the 10 nearest neighbors.  This is a non-scalable approach to retrieval.  We can't compare our query embedding with all of the news headline embeddings each time we have a query.

In [8]:
# Let's try a query for some news we might be looking for

query = 'Tiger Woods did not make the cut at golf tournament.'
query_embedding = encoder_model.encode([query])

In [9]:
# We'll start by loading all of the news embeddings into a Nearest Neighbors model

knn_model = NearestNeighbors(n_neighbors=10)
knn_model.fit(news_embeddings)

NearestNeighbors(n_neighbors=10)

In [10]:
# We'll keep track of the time it takes to find the top 10 nearest headlines

start = time.time()
dists, topk_idx = knn_model.kneighbors(query_embedding)
for d, i in zip(dists[0], topk_idx[0]):
    print(d, news_headlines[i])

print('\nTime:', time.time() - start)

# (We're using a small number of headlines so it's fast for the live session,
# but it'll still go even faster if we narrow the likely candidates first.)

0.9218926429748535 Tiger Woods missed the cut at the Farmers Insurance Open, as England's Justin Rose maintained a one-shot lead.
1.0175832509994507 English rider Guy Martin will not compete in the Ulster Grand Prix at Dundrod for the third year in a row.
1.0483298301696777 Greg Dyke will not seek re-election as Football Association chairman when his term ends in June.
1.0495320558547974 Tiger Woods admits he has concerns over the physical challenge of stepping up his return from long-term injury.
1.0528628826141357 Six-time champion Steve Davis failed to reach the World Championship as he lost 10-4 to Fergal O'Brien in the first round of qualifying in Sheffield.
1.0567675828933716 Stephen Maguire said he was "embarrassed" at not being able to motivate himself for the World Championship at the Crucible.
1.062910556793213 A golfer has suffered leg injuries after being bitten by a crocodile on an Australian golf course.
1.0718270540237427 You can't win the Davis Cup on your own and reach

Because of our small number of headlines we can get the 10 closest headlines in 9 hundredths of a second.

[Return to Top](#returnToTop)  
<a id = 'clusterRet'></a>

### 5. Retrieval via Clusters

If we can cluster the document embeddings first then we can speed up and scale the retrieval process.  We can first find clusters that are "close" to our query. Then we can actually examine (and score) all of the document embeddings within the one cluster that seems responsive to the query.

In [11]:
# Now let's try clustering the news headlines beforehand. This takes time,
# but we only need to do it once, then re-use it for different queries.

cluster_model = KMeans(n_clusters=50)
news_clusters = cluster_model.fit_predict(news_embeddings)

In [12]:
cluster_news_ids = {i: [] for i in range(50)}
for i, c in enumerate(news_clusters):
    cluster_news_ids[c].append(i)

In [13]:
# Compute the distance from the query embedding to each cluster centroid

query_cluster_dists = [cosine(query_embedding[0], cluster_model.cluster_centers_[c])
                       for c in range(50)]

In [14]:
# Get the top k nearest clusters and retrieve their document ids
# (You can try different numbers of top clusters, to see the trade-off between
# speed and recall of all the best articles we found above.)

top_clusters = np.argsort(query_cluster_dists)[:2]
candidate_news_ids = [i for c in top_clusters for i in cluster_news_ids[c]]
len(candidate_news_ids)

324

In [15]:
# Now use Nearest Neighbors only on the top cluster candidates

candidate_news_embeds = [news_embeddings[i] for i in candidate_news_ids]

knn_model = NearestNeighbors(n_neighbors=10)
knn_model.fit(candidate_news_embeds)

start = time.time()
dists, topk_idx = knn_model.kneighbors(query_embedding)
for d, i in zip(dists[0], topk_idx[0]):
    orig_i = candidate_news_ids[i]
    print(d, news_headlines[orig_i])

print('\nTime:', time.time() - start)

0.9218926429748535 Tiger Woods missed the cut at the Farmers Insurance Open, as England's Justin Rose maintained a one-shot lead.
1.0175832509994507 English rider Guy Martin will not compete in the Ulster Grand Prix at Dundrod for the third year in a row.
1.0495320558547974 Tiger Woods admits he has concerns over the physical challenge of stepping up his return from long-term injury.
1.0528628826141357 Six-time champion Steve Davis failed to reach the World Championship as he lost 10-4 to Fergal O'Brien in the first round of qualifying in Sheffield.
1.0567675828933716 Stephen Maguire said he was "embarrassed" at not being able to motivate himself for the World Championship at the Crucible.
1.0805411338806152 Wales scrum-half Rhys Webb has been ruled out of the Rugby World Cup because of a foot injury.
1.081950068473816 England's Tommy Fleetwood was denied a second 2017 European Tour title after losing a play-off to Bernd Wiesberger at the Shenzhen International.
1.0840939283370972 Leic

The clustered approach provides equally good results and it only takes less than one one hundreth of a second.  That time savings will be meaningful when we have millions or billions of records that need to be searched.

In practice, instead of the clustering approach you would want to use something like [ScaNN](https://github.com/google-research/google-research/tree/master/scann).